In [1]:
import collections
import glob
import re
import tarfile
import math
from pprint import pprint
import os
import spacy
import pandas as pd
from collections import Counter


In [2]:
def read_archive(path):
    tar = tarfile.open(path, "r:gz")   
    files = {}
    for filename in tar.getnames():
        f = tar.extractfile(filename)
        files[filename] = f.read().decode("utf-8").replace('\n', ' ').replace('\t', ' ').lower()
    return files

In [3]:
files = read_archive('./ustawy.tar.gz')

# 1. Use SpaCy tokenizer API to tokenize the text from the law corpus.

In [4]:
nlp = spacy.load('pl_core_news_sm')
docs = list(nlp.tokenizer.pipe(files.values()))

In [5]:
[tok.text for tok in docs[0][1:10]]

['dz.u', '.', 'z', '1993', 'r', '.', 'nr', '129', ',']

# 2 Compute bigram counts of downcased tokens. Given the sentence: "The quick brown fox jumps over the lazy dog.", the bigram counts are as follows: 

In [6]:
def ngrams_text(tokens, n):
    tokens = [tok.text for tok in tokens]
    return zip(*[tokens[i:] for i in range(n)])
    

In [7]:
bigrams = [bigram for doc in docs for bigram in ngrams_text(doc, 2)]
bigrams_counter = Counter(bigrams)

top_bigram_counter = bigrams_counter.most_common(20)
for text, count in top_bigram_counter:
    print(f"{text}: {count}")

('art', '.'): 83778
('ust', '.'): 53552
('poz', '.'): 45198
(',', 'poz'): 41517
('.', '1'): 36586
('-', '-'): 36542
('r', '.'): 33010
('w', 'art'): 30814
(',', 'o'): 29189
('mowa', 'w'): 27916
('w', 'ust'): 22709
(',', 'w'): 21795
('2', '.'): 21274
('1', '.'): 21111
('.', '2'): 20978
('.', 'nr'): 20786
(',', '   '): 18994
(',', 'z'): 18990
('.', ' '): 17590
('_', '_'): 16459


# 3. Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries after computing the bigram counts.

In [8]:
filtered_bigrams = Counter([bigram for bigram in bigrams if all([x.isalpha() for x in bigram])])

In [9]:
filtered_bigrams_counter = Counter(filtered_bigrams)

top_filtered_bigram_counter = filtered_bigrams_counter.most_common(20)
for text, count in top_filtered_bigram_counter:
    print(f"{text}: {count}")

('w', 'art'): 30814
('mowa', 'w'): 27916
('w', 'ust'): 22709
('których', 'mowa'): 13178
('o', 'których'): 12973
('otrzymuje', 'brzmienie'): 9530
('z', 'dnia'): 9224
('którym', 'mowa'): 8832
('o', 'którym'): 8753
('do', 'spraw'): 8350
('dodaje', 'się'): 8112
('i', 'nr'): 8055
('w', 'brzmieniu'): 6964
('w', 'drodze'): 6701
('stosuje', 'się'): 6307
('na', 'podstawie'): 6119
('w', 'przypadku'): 5687
('której', 'mowa'): 5290
('o', 'której'): 5249
('od', 'dnia'): 5079


# 4. Use pointwise mutual information to compute the measure for all pairs of words.

In [10]:
unigrams = [token.text for doc in docs for token in doc if token.is_alpha]
unigrams_count = len(unigrams)
bigrams_count = len(filtered_bigrams)
unigrams_counter = Counter(unigrams)

In [11]:
def pmi(bigram):
    P_x_y = filtered_bigrams_counter[bigram] / bigrams_count
    P_x = unigrams_counter[bigram[0]] / unigrams_count
    P_y = unigrams_counter[bigram[1]] / unigrams_count

    return math.log2(P_x_y / (P_x * P_y))

In [12]:
pmi_dict = { bigram: pmi(bigram) for bigram in filtered_bigrams_counter.keys() }
for bigram, pmi in list(pmi_dict.items())[:15]:
    print(f"{bigram}: {pmi}")

('z', 'dnia'): 7.337431793999187
('o', 'zmianie'): 8.38308839216137
('zmianie', 'ustawy'): 10.101845850117986
('ustawy', 'o'): 5.428385358916346
('o', 'podatku'): 6.648937813602218
('podatku', 'od'): 8.110998077856015
('od', 'towarów'): 7.673592765548716
('towarów', 'i'): 6.2561922531950405
('i', 'usług'): 6.559781836961522
('oraz', 'o'): 3.4509574703853705
('podatku', 'akcyzowym'): 13.287188583358178
('w', 'ustawie'): 6.888219704183044
('ustawie', 'z'): 7.70697560954953
('usług', 'oraz'): 5.81997231925886
('i', 'nr'): 5.6914531682963965


# 5. Sort the word pairs according to that measure in the descending order and determine top 10 entries.

In [13]:
top_bigram_pmi = sorted(pmi_dict.items(), key=lambda x: x[1], reverse=True)
for bigram, pmi in top_bigram_pmi[:10]:
    print(f"{bigram}: {pmi}")

('kołowe', 'jednoosiowe'): 24.629294560183173
('zbrojeń', 'żelbeto'): 24.629294560183173
('prefabrykatów', 'wnętrzowe'): 24.629294560183173
('gołe', 'aluminiowe'): 24.629294560183173
('polistyrenu', 'spienionego'): 24.629294560183173
('objaśnieniem', 'figur'): 24.629294560183173
('wkładzie', 'wnoszonym'): 24.629294560183173
('doktorem', 'habilitowanym'): 24.629294560183173
('losy', 'loteryjne'): 24.629294560183173
('uw', 'zględnieniu'): 24.629294560183173


# 6. Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5 occurrences).

In [14]:
top_bigram_pmi_5 = [(k, v) for k, v in top_bigram_pmi if filtered_bigrams_counter[k] >=5]
for bigram, pmi in top_bigram_pmi_5[:10]:
    print(f"{bigram}: {pmi}")

('świeckie', 'przygotowujące'): 22.30736646529581
('klęskami', 'żywiołowymi'): 22.30736646529581
('ręcznego', 'miotacza'): 22.30736646529581
('stajnią', 'wyścigową'): 22.30736646529581
('otworami', 'wiertniczymi'): 22.30736646529581
('obcowania', 'płciowego'): 22.30736646529581
('nietykalność', 'cielesną'): 22.30736646529581
('młyny', 'kulowe'): 22.30736646529581
('młynki', 'młotkowe'): 22.30736646529581
('obiegów', 'chłodzących'): 22.30736646529581


# 7. Use log likelihood ratio (LLR) to compute the measure for all pairs of words.

In [15]:
# Source: https://github.com/tdunning/python-llr/blob/master/llr.py

def llr_2x2(k11, k12, k21, k22):
    '''Special case of llr with a 2x2 table'''
    return 2 * (denormEntropy([k11+k12, k21+k22]) +
                denormEntropy([k11+k21, k12+k22]) -
                denormEntropy([k11, k12, k21, k22]))

def denormEntropy(counts):
    '''Computes the entropy of a list of counts scaled by the sum of the counts. If the inputs sum to one, this is just the normal definition of entropy'''
    counts = list(counts)
    total = float(sum(counts))
    # Note tricky way to avoid 0*log(0)
    return -sum([k * math.log(k/total + (k==0)) for k in counts])

In [16]:
bigram_first_token_counter = Counter()
bigram_second_token_counter = Counter()

for bigram, count in filtered_bigrams_counter.items():
    bigram_first_token_counter.update({bigram[0]: count})
    bigram_second_token_counter.update({bigram[1]: count})

bigram_to_llr = {}
for bigram, count in filtered_bigrams_counter.items():
    k11 = count
    k12 = bigram_second_token_counter[bigram[1]] - count
    k21 = bigram_first_token_counter[bigram[0]] - count
    k22 = bigrams_count - (k11 + k12 + k21)
    bigram_to_llr[bigram] = llr_2x2(k11, k12, k21, k22)

for bigram, llr in list(bigram_to_llr.items())[:15]:
    print(f"{bigram}: {llr}")

('z', 'dnia'): 19264.269867235795
('o', 'zmianie'): 4303.746595894569
('zmianie', 'ustawy'): 4929.014017024805
('ustawy', 'o'): 1417.7382283456973
('o', 'podatku'): 322.3371999912779
('podatku', 'od'): 907.0136351045512
('od', 'towarów'): 424.11969888111344
('towarów', 'i'): 510.7236186004011
('i', 'usług'): 428.1526386947371
('oraz', 'o'): 367.79739028483164
('podatku', 'akcyzowym'): 734.8974293112842
('w', 'ustawie'): 8151.568391272565
('ustawie', 'z'): 12035.801441349555
('usług', 'oraz'): 76.88171972209238
('i', 'nr'): 25650.10952515644


# 8. Sort the word pairs according to that measure in the descending order and display top 10 entries.

In [17]:
top_bigram_llr = sorted(bigram_to_llr.items(), key=lambda x: x[1], reverse=True)
for bigram, pmi in top_bigram_llr[:10]:
    print(f"{bigram}: {pmi}")

('w', 'w'): 142366.3467655643
('otrzymuje', 'brzmienie'): 85041.21853705663
('mowa', 'w'): 78603.32496332459
('których', 'mowa'): 65832.89146895928
('w', 'i'): 63911.785937641864
('w', 'z'): 52608.36599351163
('w', 'do'): 52109.980856643524
('o', 'których'): 44526.3162236776
('którym', 'mowa'): 42496.07052227837
('w', 'na'): 41572.688310367754


# 9. Compute trigram counts for the whole corpus and perform the same filtering.

In [18]:
trigrams = [trigram for doc in docs for trigram in ngrams_text(doc, 3) if all(map(lambda x: x.isalpha(), trigram))]

In [19]:
trigrams[0]

('o', 'zmianie', 'ustawy')

In [20]:
trigrams_counter = Counter(trigrams)
trigrams_count = len(trigrams)

In [21]:
for text, count in trigrams_counter.most_common(20):
    print(f"{text}: {count}")

('których', 'mowa', 'w'): 12832
('mowa', 'w', 'ust'): 12620
('o', 'których', 'mowa'): 12277
('mowa', 'w', 'art'): 11338
('którym', 'mowa', 'w'): 8663
('o', 'którym', 'mowa'): 8402
('której', 'mowa', 'w'): 5172
('o', 'której', 'mowa'): 5012
('właściwy', 'do', 'spraw'): 4482
('minister', 'właściwy', 'do'): 4440
('ustawie', 'z', 'dnia'): 3591
('w', 'ustawie', 'z'): 3589
('w', 'drodze', 'rozporządzenia'): 3466
('ustawy', 'z', 'dnia'): 2819
('dodaje', 'się', 'ust'): 2753
('stosuje', 'się', 'odpowiednio'): 2672
('zastępuje', 'się', 'wyrazami'): 2379
('wejścia', 'w', 'życie'): 2204
('dni', 'od', 'dnia'): 1936
('wprowadza', 'się', 'następujące'): 1752


# 10. Use PMI (with 5 occurrence threshold) and LLR to compute top 10 results for the trigrams. Devise a method for computing the values, based on the results for bigrams.

In [22]:
trigrams_count=len(trigrams)
def pmi3(trigram):
    P_x_y_z = trigrams_counter[trigram] / trigrams_count
    P_x = unigrams_counter[trigram[0]] / unigrams_count
    P_y = unigrams_counter[trigram[1]] / unigrams_count
    P_z = unigrams_counter[trigram[2]] / unigrams_count

    return math.log2(P_x_y_z / (P_x * P_y * P_z))

In [23]:
pmi_dict_3 = { trigram: pmi3(trigram) for trigram in trigrams_counter.keys() }
for trigram, pmi in list(pmi_dict_3.items())[:15]:
    print(f"{trigram}: {pmi}")

('o', 'zmianie', 'ustawy'): 13.916328572230125
('zmianie', 'ustawy', 'o'): 13.506477519666616
('ustawy', 'o', 'podatku'): 10.75939188577996
('o', 'podatku', 'od'): 10.266079911635046
('podatku', 'od', 'towarów'): 16.13152288350617
('od', 'towarów', 'i'): 10.878285993885402
('towarów', 'i', 'usług'): 15.044682361138472
('oraz', 'o', 'podatku'): 8.097337246260917
('o', 'podatku', 'akcyzowym'): 16.989795736694937
('w', 'ustawie', 'z'): 9.883092775592244
('ustawie', 'z', 'dnia'): 13.37014960530769
('i', 'usług', 'oraz'): 8.819738031604635
('usług', 'oraz', 'o'): 8.536500729288564
('wprowadza', 'się', 'następujące'): 17.614125814370496
('się', 'następujące', 'zmiany'): 16.60269580119262


In [24]:
top_trigram_pmi = sorted(pmi_dict_3.items(), key=lambda x: x[1], reverse=True)
for trigram, pmi in top_trigram_pmi[:10]:
    print(f"{trigram}: {pmi}")

('english', 'language', 'college'): 44.44245278218417
('wniebowzięcia', 'najświętszej', 'maryi'): 44.44245278218417
('najświętszej', 'maryi', 'panny'): 44.44245278218417
('porejestrowe', 'doświadczalnictwo', 'odmianowe'): 44.44245278218417
('world', 'jewish', 'restitution'): 44.44245278218417
('jewish', 'restitution', 'organisation'): 44.44245278218417
('wszywanie', 'zamków', 'błyskawicznych'): 44.44245278218417
('mit', 'beschrankter', 'haftung'): 44.44245278218417
('prosimy', 'uważnie', 'przeczytać'): 44.44245278218417
('aegroti', 'suprema', 'lex'): 44.44245278218417


In [25]:
top_trigram_pmi_5 = [(k, v) for k, v in top_trigram_pmi if trigrams_counter[k] >=5]
for trigram, pmi in top_trigram_pmi_5[:10]:
    print(f"{trigram}: {pmi}")

('finałowego', 'turnieju', 'mistrzostw'): 37.282581445405775
('profilem', 'zaufanym', 'epuap'): 37.04157334590198
('centralnemu', 'biuru', 'antykorupcyjnemu'): 36.590703740768106
('turnieju', 'mistrzostw', 'europy'): 36.5170466990428
('potwierdzonym', 'profilem', 'zaufanym'): 36.494085550599486
('przedwczesnego', 'wyrębu', 'drzewostanu'): 36.37206345429277
('piłce', 'nożnej', 'uefa'): 36.34652836218563
('cienką', 'sierścią', 'zwierzęcą'): 35.84551763979693
('szybkiemu', 'postępowi', 'technicznemu'): 35.780674684412176
('wypalonym', 'paliwem', 'jądrowym'): 35.72820726451804


In [26]:
trigram_first_token_counter = Counter()
trigram_second_token_counter = Counter()

for trigram, count in trigrams_counter.items():
    trigram_first_token_counter.update({trigram[0]+'/'+trigram[1]: count})
    trigram_second_token_counter.update({trigram[1]+'/'+trigram[2]: count})

trigram_to_llr = {}
for trigram, count in trigrams_counter.items():
    bigram = [trigram[0]+'/'+trigram[1], trigram[1]+'/'+trigram[2]]
    k11 = count
    k12 = trigram_second_token_counter[bigram[1]] - count
    k21 = trigram_first_token_counter[bigram[0]] - count
    k22 = trigrams_count - (k11 + k12 + k21)
    trigram_to_llr[trigram] = llr_2x2(k11, k12, k21, k22)

for trigram, llr in list(trigram_to_llr.items())[:15]:
    print(f"{trigram}: {llr}")

('o', 'zmianie', 'ustawy'): 12987.959317659846
('zmianie', 'ustawy', 'o'): 9849.325623455821
('ustawy', 'o', 'podatku'): 1506.0660337779118
('o', 'podatku', 'od'): 1493.21630260675
('podatku', 'od', 'towarów'): 3336.0298646069296
('od', 'towarów', 'i'): 3854.644990233668
('towarów', 'i', 'usług'): 6572.828875478961
('oraz', 'o', 'podatku'): 542.4027405116612
('o', 'podatku', 'akcyzowym'): 1060.793556188808
('w', 'ustawie', 'z'): 47862.47038487035
('ustawie', 'z', 'dnia'): 41486.129270834484
('i', 'usług', 'oraz'): 1557.8588251239817
('usług', 'oraz', 'o'): 712.3141234022551
('wprowadza', 'się', 'następujące'): 27176.535077181317
('się', 'następujące', 'zmiany'): 27488.976115050507


In [27]:
top_trigram_llr = sorted(trigram_to_llr.items(), key=lambda x: x[1], reverse=True)
for trigram, pmi in top_trigram_llr[:10]:
    print(f"{trigram}: {pmi}")

('o', 'których', 'mowa'): 147964.73903079642
('których', 'mowa', 'w'): 116364.41917255323
('o', 'którym', 'mowa'): 107692.39306950809
('mowa', 'w', 'ust'): 103501.24431163131
('mowa', 'w', 'art'): 82319.81622264214
('którym', 'mowa', 'w'): 76811.98861773114
('o', 'której', 'mowa'): 69316.76684738755
('minister', 'właściwy', 'do'): 60975.49989174932
('właściwy', 'do', 'spraw'): 50572.20357742673
('w', 'ustawie', 'z'): 47862.47038487035


# 11 Create a table comparing the methods (separate table for bigrams and trigrams).

## Bigrams

In [28]:
columns = ['Top occurences', 'PMI', 'PMI >= 5', 'LLR']
data=[]
for i in range(10):
    data.append((' '.join(top_filtered_bigram_counter[i][0]),
                ' '.join(top_bigram_pmi[i][0]),
                ' '.join(top_bigram_pmi_5[i][0]),
                ' '.join(top_bigram_llr[i][0])))

In [29]:
pd.DataFrame(data, columns=columns)

,Top occurences,PMI,PMI >= 5,LLR
0,w art,kołowe jednoosiowe,świeckie przygotowujące,w w
1,mowa w,zbrojeń żelbeto,klęskami żywiołowymi,otrzymuje brzmienie
2,w ust,prefabrykatów wnętrzowe,ręcznego miotacza,mowa w
3,których mowa,gołe aluminiowe,stajnią wyścigową,których mowa
4,o których,polistyrenu spienionego,otworami wiertniczymi,w i
5,otrzymuje brzmienie,objaśnieniem figur,obcowania płciowego,w z
6,z dnia,wkładzie wnoszonym,nietykalność cielesną,w do
7,którym mowa,doktorem habilitowanym,młyny kulowe,o których
8,o którym,losy loteryjne,młynki młotkowe,którym mowa
9,do spraw,uw zględnieniu,obiegów chłodzących,w na


## Trigrams

In [30]:
columns = ['Top occurences', 'PMI', 'PMI >= 5', 'LLR']
data=[]
top_trigram_counter=trigrams_counter.most_common(20)
for i in range(10):
    data.append((' '.join(top_trigram_counter[i][0]),
                ' '.join(top_trigram_pmi[i][0]),
                ' '.join(top_trigram_pmi_5[i][0]),
                ' '.join(top_trigram_llr[i][0])))

In [31]:
pd.DataFrame(data, columns=columns)

,Top occurences,PMI,PMI >= 5,LLR
0,których mowa w,english language college,finałowego turnieju mistrzostw,o których mowa
1,mowa w ust,wniebowzięcia najświętszej maryi,profilem zaufanym epuap,których mowa w
2,o których mowa,najświętszej maryi panny,centralnemu biuru antykorupcyjnemu,o którym mowa
3,mowa w art,porejestrowe doświadczalnictwo odmianowe,turnieju mistrzostw europy,mowa w ust
4,którym mowa w,world jewish restitution,potwierdzonym profilem zaufanym,mowa w art
5,o którym mowa,jewish restitution organisation,przedwczesnego wyrębu drzewostanu,którym mowa w
6,której mowa w,wszywanie zamków błyskawicznych,piłce nożnej uefa,o której mowa
7,o której mowa,mit beschrankter haftung,cienką sierścią zwierzęcą,minister właściwy do
8,właściwy do spraw,prosimy uważnie przeczytać,szybkiemu postępowi technicznemu,właściwy do spraw
9,minister właściwy do,aegroti suprema lex,wypalonym paliwem jądrowym,w ustawie z


# 12. Answer the following questions:
### Why do we have to filter the bigrams, rather than the token sequence?
Example: `a b c d` - bigrams: `(a,b), (b,c), (c,d)`. If we now filter b in token sequence, we will get `(a,c), (c,d)` even if `a` was not neighbour of `c`. We modify context
### Which measure (PMI, PMI with filtering, LLR) works better for the bigrams and which for the trigrams?
It depends - PMI without filtering gives mostly proper names or some very specific collocations which gives no to very little info about structure of the analyzed text. The filtering causes it to return some concepts that are rare, but appear some times (eg. `wypalonym paliwem jądrowym `, `ręcznego miotacza`. It gives wide context about what we can expect from the dataset. 
The LLR method gives most information about structure of the text. we can see clearly from it that it is a law text.
### What types of expressions are discovered by the methods.
As said earlier, LLR is returning results more correlated to corpus. PMI returns very specific collocations and proper names giving no to very little info about corpus. 
### Can you devise a different type of filtering that would yield better results?
We can try to lemmatize words before to minimize amount of very likely expressions. We can also think about filtering prepositions.